In [8]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "Rohan@556"))

def create_node(driver, node_label, node_name):
    with driver.session() as session:
        cypher_query = f"CREATE (n:{node_label} {{name: $name}})"
        
        session.run(cypher_query, name=node_name)

create_node(driver, "Person", "Alice")


In [9]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "Rohan@556"))

def add_files_to_user(driver, user_id, file1, file2):
    with driver.session() as session:
        cypher_query = """
        MERGE (u:User {id: $user_id})
        SET u.encryptedFile = $file1, u.keyFile = $file2
        """
        session.run(cypher_query, user_id=user_id, file1=file1, file2=file2)

file1 = "C:/Users/ROHAN/IotSimulation/Neo4j/encrypted_user_1_auth_EMP019_EMP017_merged_1"
file2 = "C:/Users/ROHAN/IotSimulation/Neo4j/key_iv_user_1_auth_EMP019_EMP017_merged_1.txt.bin"

add_files_to_user(driver, "user_1", file1, file2)

driver.close()


In [11]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "Rohan@556"))

def create_file_nodes_and_link_to_user(driver, user_id, file1, file2):
    with driver.session() as session:
        cypher_query = """
        MERGE (u:User {id: $user_id})
        CREATE (f1:File {path: $file1})
        CREATE (f2:File {path: $file2})
        MERGE (u)-[:HAS_FILE]->(f1)
        MERGE (u)-[:HAS_FILE]->(f2)
        """
        session.run(cypher_query, user_id=user_id, file1=file1, file2=file2)

file1 = "C:/Users/ROHAN/IotSimulation/Neo4j/encrypted_user_1_auth_EMP019_EMP017_merged_1"
file2 = "C:/Users/ROHAN/IotSimulation/Neo4j/key_iv_user_1_auth_EMP019_EMP017_merged_1.txt.bin"

create_file_nodes_and_link_to_user(driver, "user_1", file1, file2)
driver.close()


In [13]:
import os
from neo4j import GraphDatabase

def list_files_in_directory(directory):
    """List all files in a given directory."""
    return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

def extract_user_id(file_name):
    """Extract user ID from the file name based on the provided naming convention."""
    parts = file_name.split('_')
    if len(parts) > 1:
        return 'user_' + parts[1] 
    return None

def upload_files_to_neo4j(driver, directory):
    files = list_files_in_directory(directory)
    for file in files:
        user_id = extract_user_id(file)
        if user_id:
            file_path = os.path.join(directory, file)
            add_file_to_user(driver, user_id, file_path)

def add_file_to_user(driver, user_id, file_path):
    with driver.session() as session:
        cypher_query = """
        MERGE (u:User {id: $user_id})
        MERGE (f:File {path: $file_path})
        MERGE (u)-[:HAS_FILE]->(f)
        """
        session.run(cypher_query, user_id=user_id, file_path=file_path)

uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "Rohan@556"))
directories = ["C:/Users/ROHAN/IotSimulation/AES_enc", "C:/Users/ROHAN/IotSimulation/AES_Keys"]
for directory in directories:
    upload_files_to_neo4j(driver, directory)

driver.close()


In [27]:
from neo4j import GraphDatabase
import os

class GraphDBManager:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_user_with_file_paths(self, user_name, file_paths):
        with self.driver.session() as session:
            session.write_transaction(self._create_and_link, user_name, file_paths)

    @staticmethod
    def _create_and_link(tx, user_name, file_paths):
        tx.run("MERGE (user:User {name: $user_name})", user_name=user_name)
        
        for file_path in file_paths:
            file_name = os.path.basename(file_path)
            tx.run(
                "MATCH (user:User {name: $user_name}) "
                "MERGE (file:File {name: $file_name, path: $file_path}) "
                "MERGE (user)-[:CONTAINS]->(file)",
                user_name=user_name, file_name=file_name, file_path=file_path
            )

uri = "neo4j://localhost:7687"
driver = GraphDBManager(uri, "neo4j", "Rohan@556")

file_paths = [
    "C:/Users/ROHAN/IotSimulation/Neo4j/encrypted_user_1_auth_EMP019_EMP017_merged_1",
    "C:/Users/ROHAN/IotSimulation/Neo4j/key_iv_user_1_auth_EMP019_EMP017_merged_1.txt.bin"
]
user_name = "user_1"
driver.create_user_with_file_paths(user_name, file_paths)
driver.close()


C:\Users\ROHAN\AppData\Local\Temp\ipykernel_9872\1789062553.py:13: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link, user_name, file_paths)


In [ ]:
"C:/Users/ROHAN/IotSimulation/AES_enc"

In [ ]:
"C:/Users/ROHAN/IotSimulation/AES_Keys"

# Clear Database

In [16]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "Rohan@556"))

def delete_all_data(driver):
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")
        print("All nodes and relationships have been deleted.")
delete_all_data(driver)
driver.close()


All nodes and relationships have been deleted.


# Exporting all key and files to the graph database

In [15]:
import os
from neo4j import GraphDatabase

class GraphDBManager:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_users_with_files(self, folder_path1, folder_path2):
        with self.driver.session() as session:
            for file_path in os.listdir(folder_path1):
                full_path = os.path.join(folder_path1, file_path)
                user_name = self.extract_user_from_filename(file_path)
                session.write_transaction(self._create_and_link, user_name, full_path)

            for file_path in os.listdir(folder_path2):
                full_path = os.path.join(folder_path2, file_path)
                user_name = self.extract_user_from_filename(file_path)
                session.write_transaction(self._create_and_link, user_name, full_path)

    def extract_user_from_filename(self, filename):
        parts = filename.split('_')
        user_index = parts.index('user') + 1
        return 'user_' + parts[user_index]

    @staticmethod
    def _create_and_link(tx, user_name, file_path):
        file_name = os.path.basename(file_path)
        tx.run("MERGE (user:User {name: $user_name})", user_name=user_name)
        tx.run(
            "MATCH (user:User {name: $user_name}) "
            "MERGE (file:File {name: $file_name, path: $file_path}) "
            "MERGE (user)-[:CONTAINS]->(file)",
            user_name=user_name, file_name=file_name, file_path=file_path
        )

uri = "neo4j://localhost:7687"

driver = GraphDBManager(uri, "neo4j", "Rohan@556")

folder_path1 = "C:/Users/ROHAN/IotSimulation/OneHour/50KB/50KBTrials/TRIAL[2]50kb/50KB_AES_Enc"
folder_path2 = "C:/Users/ROHAN/IotSimulation/OneHour/50KB/50KBTrials/TRIAL[2]50kb/50KB_AES_Enc_key"

driver.create_users_with_files(folder_path1, folder_path2)
driver.close()


C:\Users\ROHAN\AppData\Local\Temp\ipykernel_7540\4061915418.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link, user_name, full_path)
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_7540\4061915418.py:21: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link, user_name, full_path)


In [ ]:
C:/Users/ROHAN/IotSimulation/OneHour/50KB/50KBTrials/TRIAL[1]50kb/50KB_AES_Enc

In [ ]:
C:/Users/ROHAN/IotSimulation/OneHour/50KB/50KBTrials/TRIAL[1]50kb/50KB_AES_Enc_key

# Optimized code below

In [35]:
import os
from neo4j import GraphDatabase

class GraphDBManager:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.create_indexes()  # Create indexes when initializing the DB connection

    def close(self):
        self.driver.close()

    def create_indexes(self):
        with self.driver.session() as session:
            session.write_transaction(self._create_user_index)
            session.write_transaction(self._create_file_index)

    @staticmethod
    def _create_user_index(tx):
        tx.run("CREATE INDEX user_name_index IF NOT EXISTS FOR (u:User) ON (u.name)")

    @staticmethod
    def _create_file_index(tx):
        tx.run("CREATE INDEX file_name_index IF NOT EXISTS FOR (f:File) ON (f.name)")

    def create_users_with_files(self, folder_path1, folder_path2):
        with self.driver.session() as session:
            for file_path in os.listdir(folder_path1):
                full_path = os.path.join(folder_path1, file_path)
                user_name = self.extract_user_from_filename(file_path)
                session.write_transaction(self._create_and_link, user_name, full_path)

            for file_path in os.listdir(folder_path2):
                full_path = os.path.join(folder_path2, file_path)
                user_name = self.extract_user_from_filename(file_path)
                session.write_transaction(self._create_and_link, user_name, full_path)

    def extract_user_from_filename(self, filename):
        parts = filename.split('_')
        user_index = parts.index('user') + 1
        return 'user_' + parts[user_index]

    @staticmethod
    def _create_and_link(tx, user_name, file_path):
        file_name = os.path.basename(file_path)
        tx.run("MERGE (user:User {name: $user_name})", user_name=user_name)
        tx.run(
            "MATCH (user:User {name: $user_name}) "
            "MERGE (file:File {name: $file_name, path: $file_path}) "
            "MERGE (user)-[:CONTAINS]->(file)",
            user_name=user_name, file_name=file_name, file_path=file_path
        )

# Usage example
uri = "neo4j://localhost:7687"
driver = GraphDBManager(uri, "neo4j", "Rohan@556")

folder_path1 = "C:/Users/ROHAN/IotSimulation/OneHour/50KB/50KBTrials/TRIAL[2]50kb/50KB_AES_Enc"
folder_path2 = "C:/Users/ROHAN/IotSimulation/OneHour/50KB/50KBTrials/TRIAL[2]50kb/50KB_AES_Enc_key"

driver.create_users_with_files(folder_path1, folder_path2)
driver.close()


C:\Users\ROHAN\AppData\Local\Temp\ipykernel_7540\2822232708.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_user_index)
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_7540\2822232708.py:15: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_file_index)
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_7540\2822232708.py:30: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link, user_name, full_path)
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_7540\2822232708.py:35: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link, user_name, full_path)


# 2nd Optimization

In [2]:
import os
from neo4j import GraphDatabase

class GraphDBManager:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.create_constraints_and_indexes()  # Initialize constraints and indexes when creating the DB connection

    def close(self):
        self.driver.close()

    def create_constraints_and_indexes(self):
        with self.driver.session() as session:
            # Drop the existing index if it exists to avoid conflict with the unique constraint
            session.write_transaction(self._drop_user_index_if_exists)
            # Create a unique constraint on User.name
            session.write_transaction(self._create_user_name_constraint)
            # Create an index on File.name
            session.write_transaction(self._create_file_index)

    @staticmethod
    def _drop_user_index_if_exists(tx):
        tx.run("DROP INDEX user_name_index IF EXISTS")

    @staticmethod
    def _create_user_name_constraint(tx):
        tx.run("CREATE CONSTRAINT user_name_constraint IF NOT EXISTS FOR (u:User) REQUIRE u.name IS UNIQUE")

    @staticmethod
    def _create_file_index(tx):
        tx.run("CREATE INDEX file_name_index IF NOT EXISTS FOR (f:File) ON (f.name)")

    def create_users_with_files(self, folder_path1, folder_path2):
        with self.driver.session() as session:
            for file_path in os.listdir(folder_path1):
                full_path = os.path.join(folder_path1, file_path)
                user_name = self.extract_user_from_filename(file_path)
                session.write_transaction(self._create_and_link, user_name, full_path)

            for file_path in os.listdir(folder_path2):
                full_path = os.path.join(folder_path2, file_path)
                user_name = self.extract_user_from_filename(file_path)
                session.write_transaction(self._create_and_link, user_name, full_path)

    def extract_user_from_filename(self, filename):
        parts = filename.split('_')
        user_index = parts.index('user') + 1
        return 'user_' + parts[user_index]

    @staticmethod
    def _create_and_link(tx, user_name, file_path):
        file_name = os.path.basename(file_path)
        tx.run(
            "MERGE (user:User {name: $user_name}) "
            "MERGE (file:File {name: $file_name, path: $file_path}) "
            "MERGE (user)-[:CONTAINS]->(file)",
            user_name=user_name, file_name=file_name, file_path=file_path
        )

# Usage example
uri = "neo4j://localhost:7687"
driver = GraphDBManager(uri, "neo4j", "Rohan@556")

folder_path1 = "C:/Users/ROHAN/IotSimulation/AES_enc"
folder_path2 = "C:/Users/ROHAN/IotSimulation/AES_Keys"

driver.create_users_with_files(folder_path1, folder_path2)
driver.close()

C:\Users\ROHAN\AppData\Local\Temp\ipykernel_15664\4020208029.py:15: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._drop_user_index_if_exists)
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_15664\4020208029.py:17: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_user_name_constraint)
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_15664\4020208029.py:19: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_file_index)
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_15664\4020208029.py:38: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link, user_name, full_path)
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_15664\4020208029.py:43: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link,

# To get the data associated with user

In [10]:
import os
import time
from neo4j import GraphDatabase

class GraphDBManager:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_users_with_files(self, folder_path1, folder_path2):
        with self.driver.session() as session:
            for file_path in os.listdir(folder_path1):
                full_path = os.path.join(folder_path1, file_path)
                user_name = self.extract_user_from_filename(file_path)
                session.write_transaction(self._create_and_link, user_name, full_path)

            for file_path in os.listdir(folder_path2):
                full_path = os.path.join(folder_path2, file_path)
                user_name = self.extract_user_from_filename(file_path)
                session.write_transaction(self._create_and_link, user_name, full_path)

    def extract_user_from_filename(self, filename):
        parts = filename.split('_')
        user_index = parts.index('user') + 1
        return 'user_' + parts[user_index]

    @staticmethod
    def _create_and_link(tx, user_name, file_path):
        file_name = os.path.basename(file_path)
        tx.run("MERGE (user:User {name: $user_name})", user_name=user_name)
        tx.run(
            "MATCH (user:User {name: $user_name}) "
            "MERGE (file:File {name: $file_name, path: $file_path}) "
            "MERGE (user)-[:CONTAINS]->(file)",
            user_name=user_name, file_name=file_name, file_path=file_path
        )

    def get_user_info(self, user_name):
        with self.driver.session() as session:
            start_time = time.time()
            result = session.read_transaction(self._find_user_info, user_name)
            end_time = time.time()
            time_taken = end_time - start_time  
            return result, time_taken

    @staticmethod
    def _find_user_info(tx, user_name):
        query = (
            "MATCH (user:User {name: $user_name})-[:CONTAINS]->(file) "
            "RETURN user, collect(file) as files"
        )
        result = tx.run(query, user_name=user_name)
        return [record for record in result]

uri = "neo4j://localhost:7687"
driver = GraphDBManager(uri, "neo4j", "Rohan@556")

folder_path1 = "C:/Users/ROHAN/IotSimulation/AES_enc"
folder_path2 = "C:/Users/ROHAN/IotSimulation/AES_Keys"
driver.create_users_with_files(folder_path1, folder_path2)

user_info, time_taken = driver.get_user_info("user_4")
print(f"Time taken: {time_taken} seconds")
print("User Info:", user_info)

driver.close()

C:\Users\ROHAN\AppData\Local\Temp\ipykernel_2360\2916422048.py:17: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link, user_name, full_path)
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_2360\2916422048.py:22: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link, user_name, full_path)
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_2360\2916422048.py:43: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._find_user_info, user_name)


Time taken: 2.0726256370544434 seconds
User Info: [<Record user=<Node element_id='4:9b702522-6b85-40f7-afb4-81c1efa057cf:86' labels=frozenset({'User'}) properties={'name': 'user_4'}> files=[<Node element_id='4:9b702522-6b85-40f7-afb4-81c1efa057cf:143' labels=frozenset({'File'}) properties={'path': 'C:/Users/ROHAN/IotSimulation/AES_Keys\\key_iv_user_4_auth_EMP012_EMP001_merged_1.txt.bin', 'name': 'key_iv_user_4_auth_EMP012_EMP001_merged_1.txt.bin'}>, <Node element_id='4:9b702522-6b85-40f7-afb4-81c1efa057cf:87' labels=frozenset({'File'}) properties={'path': 'C:/Users/ROHAN/IotSimulation/AES_enc\\encrypted_user_4_auth_EMP012_EMP001_merged_1.txt', 'name': 'encrypted_user_4_auth_EMP012_EMP001_merged_1.txt'}>]>]


# Code to run on GraphDB to get user data

In [ ]:
MATCH (user:User {name: 'user_123'})-[:CONTAINS]->(file)
RETURN user, collect(file) as files

# Code to get all the user who have affected by any data userID

In [12]:
import os
import time
from neo4j import GraphDatabase

class GraphDBManager:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_users_with_files(self, folder_path1, folder_path2):
        with self.driver.session() as session:
            for file_path in os.listdir(folder_path1):
                full_path = os.path.join(folder_path1, file_path)
                user_name = self.extract_user_from_filename(file_path)
                session.write_transaction(self._create_and_link, user_name, full_path)

            for file_path in os.listdir(folder_path2):
                full_path = os.path.join(folder_path2, file_path)
                user_name = self.extract_user_from_filename(file_path)
                session.write_transaction(self._create_and_link, user_name, full_path)

    def extract_user_from_filename(self, filename):
        parts = filename.split('_')
        user_index = parts.index('user') + 1
        return 'user_' + parts[user_index]

    @staticmethod
    def _create_and_link(tx, user_name, file_path):
        file_name = os.path.basename(file_path)
        tx.run("MERGE (user:User {name: $user_name})", user_name=user_name)
        tx.run(
            "MATCH (user:User {name: $user_name}) "
            "MERGE (file:File {name: $file_name, path: $file_path}) "
            "MERGE (user)-[:CONTAINS]->(file)",
            user_name=user_name, file_name=file_name, file_path=file_path
        )

    def get_user_info(self, user_name):
        with self.driver.session() as session:
            start_time = time.time()
            result = session.read_transaction(self._find_user_info, user_name)
            end_time = time.time()
            time_taken = end_time - start_time  
            return result, time_taken

    @staticmethod
    def _find_user_info(tx, user_name):
        query = (
            "MATCH (user:User {name: $user_name})-[:CONTAINS]->(file) "
            "RETURN user, collect(file) as files"
        )
        result = tx.run(query, user_name=user_name)
        return [record for record in result]

    def count_users_by_filename_keyword(self, keyword):
        with self.driver.session() as session:
            start_time = time.time()
            result = session.read_transaction(self._count_users_by_keyword, keyword)
            end_time = time.time()
            time_taken = end_time - start_time  
            return result, time_taken

    @staticmethod
    def _count_users_by_keyword(tx, keyword):
        query = (
            "MATCH (user:User)-[:CONTAINS]->(file:File) "
            "WHERE file.name CONTAINS $keyword "
            "RETURN COUNT(DISTINCT user) AS userCount"
        )
        result = tx.run(query, keyword=keyword).single().value()
        return result

# Usage example
uri = "neo4j://localhost:7687"
driver = GraphDBManager(uri, "neo4j", "Rohan@556")

# Create users and files (assuming these folders contain your files)
folder_path1 = "C:/Users/ROHAN/IotSimulation/OneHour/50KB/50KBTrials/TRIAL[1]50kb/50KB_AES_Enc"
folder_path2 = "C:/Users/ROHAN/IotSimulation/OneHour/50KB/50KBTrials/TRIAL[1]50kb/50KB_AES_Enc_key"
driver.create_users_with_files(folder_path1, folder_path2)

# Count users with 'EMP008' in their file names
users_count, time_taken = driver.count_users_by_filename_keyword("EMP003")
print(f"Time taken: {time_taken} seconds")
print(f"Number of users with 'EMP008' in their filenames: {users_count}")

# Retrieve specific user info if needed
# user_info, time_taken = driver.get_user_info("user_4")
# print(f"Time taken: {time_taken} seconds")
# print("User Info:", user_info)

driver.close()


C:\Users\ROHAN\AppData\Local\Temp\ipykernel_17240\4026940138.py:17: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link, user_name, full_path)
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_17240\4026940138.py:22: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link, user_name, full_path)
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_17240\4026940138.py:60: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._count_users_by_keyword, keyword)


Time taken: 2.0544304847717285 seconds
Number of users with 'EMP008' in their filenames: 10


In [16]:
import os
import time
from neo4j import GraphDatabase

class GraphDBManager:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_users_with_files(self, folder_path1, folder_path2):
        with self.driver.session() as session:
            for file_path in os.listdir(folder_path1):
                full_path = os.path.join(folder_path1, file_path)
                user_name = self.extract_user_from_filename(file_path)
                session.write_transaction(self._create_and_link, user_name, full_path)

            for file_path in os.listdir(folder_path2):
                full_path = os.path.join(folder_path2, file_path)
                user_name = self.extract_user_from_filename(file_path)
                session.write_transaction(self._create_and_link, user_name, full_path)

    def extract_user_from_filename(self, filename):
        parts = filename.split('_')
        user_index = parts.index('user') + 1
        return 'user_' + parts[user_index]

    @staticmethod
    def _create_and_link(tx, user_name, file_path):
        file_name = os.path.basename(file_path)
        tx.run("MERGE (user:User {name: $user_name})", user_name=user_name)
        tx.run(
            "MATCH (user:User {name: $user_name}) "
            "MERGE (file:File {name: $file_name, path: $file_path}) "
            "MERGE (user)-[:CONTAINS]->(file)",
            user_name=user_name, file_name=file_name, file_path=file_path
        )

    def get_users_and_files_by_keyword(self, keyword):
        with self.driver.session() as session:
            start_time = time.time()
            result = session.read_transaction(self._find_users_and_files_by_keyword, keyword)
            end_time = time.time()
            time_taken = end_time - start_time
            user_count = len(set([record['user']['name'] for record in result]))
            return result, user_count, time_taken

    @staticmethod
    def _find_users_and_files_by_keyword(tx, keyword):
        query = (
            "MATCH (user:User)-[:CONTAINS]->(file:File) "
            "WHERE file.name CONTAINS $keyword "
            "RETURN user, file.name AS fileName"
        )
        result = tx.run(query, keyword=keyword)
        return [record for record in result]

# Usage example
uri = "neo4j://localhost:7687"
driver = GraphDBManager(uri, "neo4j", "Rohan@556")

# Assuming you've already created users and their files
users_files, user_count, time_taken = driver.get_users_and_files_by_keyword("EMP058")
print(f"Time taken: {time_taken} seconds")
print(f"Number of unique users with 'EMP008' in their filenames: {user_count}")
for record in users_files:
    print(f"User: {record['user']['name']}, File: {record['fileName']}")

driver.close()


C:\Users\ROHAN\AppData\Local\Temp\ipykernel_7540\1765769980.py:43: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._find_users_and_files_by_keyword, keyword)


Time taken: 4.083089351654053 seconds
Number of unique users with 'EMP008' in their filenames: 6
User: user_100, File: encrypted_user_100_auth_EMP031_EMP058_EMP068_EMP050_aggregated.txt
User: user_19, File: encrypted_user_19_auth_EMP025_EMP058_EMP026_EMP072_EMP059_EMP008_aggregated.txt
User: user_31, File: encrypted_user_31_auth_EMP058_EMP017_EMP073_EMP045_EMP041_EMP046_EMP030_EMP009_EMP040_aggregated.txt
User: user_41, File: encrypted_user_41_auth_EMP080_EMP041_EMP076_EMP074_EMP018_EMP058_aggregated.txt
User: user_56, File: encrypted_user_56_auth_EMP013_EMP083_EMP058_EMP003_EMP024_EMP017_EMP063_EMP020_EMP002_aggregated.txt
User: user_88, File: encrypted_user_88_auth_EMP058_EMP038_aggregated.txt
User: user_100, File: key_user_100_auth_EMP031_EMP058_EMP068_EMP050_aggregated.txt.bin
User: user_19, File: key_user_19_auth_EMP025_EMP058_EMP026_EMP072_EMP059_EMP008_aggregated.txt.bin
User: user_31, File: key_user_31_auth_EMP058_EMP017_EMP073_EMP045_EMP041_EMP046_EMP030_EMP009_EMP040_aggregat

# Optimize Verison

In [29]:
import os
from neo4j import GraphDatabase

class GraphDBManager:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.create_indexes()

    def close(self):
        self.driver.close()

    def create_indexes(self):
        with self.driver.session() as session:
            session.write_transaction(self._create_user_index)
            session.write_transaction(self._create_file_index)

    @staticmethod
    def _create_user_index(tx):
        tx.run("CREATE INDEX user_name_index IF NOT EXISTS FOR (u:User) ON (u.name)")

    @staticmethod
    def _create_file_index(tx):
        tx.run("CREATE INDEX file_name_index IF NOT EXISTS FOR (f:File) ON (f.name)")

    def create_users_with_files(self, folder_paths):
        for folder_path in folder_paths:
            self._process_folder(folder_path)

    def _process_folder(self, folder_path):
        with self.driver.session() as session:
            for file_path in os.listdir(folder_path):
                full_path = os.path.join(folder_path, file_path)
                user_name = self.extract_user_from_filename(file_path)
                session.write_transaction(self._create_and_link, user_name, full_path)

    @staticmethod
    def extract_user_from_filename(filename):
        parts = filename.split('_')
        user_index = parts.index('user') + 1
        return 'user_' + parts[user_index]

    @staticmethod
    def _create_and_link(tx, user_name, file_path):
        file_name = os.path.basename(file_path)
        query = """
        MERGE (user:User {name: $user_name})
        MERGE (file:File {name: $file_name, path: $file_path})
        MERGE (user)-[:CONTAINS]->(file)
        """
        tx.run(query, user_name=user_name, file_name=file_name, file_path=file_path)

    def get_files_by_keyword(self, keyword):
        query = """
        MATCH (user:User)-[:CONTAINS]->(file:File)
        WHERE file.name CONTAINS $keyword
        RETURN user.name AS userName, file.name AS fileName
        """
        with self.driver.session() as session:
            start_time = time.time()
            result = session.run(query, keyword=keyword)
            records = [record for record in result]
            end_time = time.time()
            time_taken = end_time - start_time
            return records, time_taken

# Usage example
uri = "neo4j://localhost:7687"
driver = GraphDBManager(uri, "neo4j", "Rohan@556")

folder_paths = [
    "C:/Users/ROHAN/IotSimulation/OneHour/50KB/50KBTrials/TRIAL[2]50kb/50KB_AES_Enc",
    "C:/Users/ROHAN/IotSimulation/OneHour/50KB/50KBTrials/TRIAL[2]50kb/50KB_AES_Enc_key"
]
driver.create_users_with_files(folder_paths)

keyword = "EMP012"
files, time_taken = driver.get_files_by_keyword(keyword)
print(f"Time taken for retrieval: {time_taken} seconds")
for record in files:
    print(f"User: {record['userName']}, File: {record['fileName']}")

driver.close()


C:\Users\ROHAN\AppData\Local\Temp\ipykernel_7540\3092586938.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_user_index)
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_7540\3092586938.py:15: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_file_index)
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_7540\3092586938.py:34: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link, user_name, full_path)


Time taken for retrieval: 2.156001329421997 seconds
User: user_15, File: encrypted_user_15_auth_EMP072_EMP081_EMP056_EMP040_EMP016_EMP039_EMP012_EMP059_EMP076_EMP078_aggregated.txt
User: user_52, File: encrypted_user_52_auth_EMP075_EMP022_EMP038_EMP012_EMP024_EMP060_EMP042_EMP040_aggregated.txt
User: user_54, File: encrypted_user_54_auth_EMP076_EMP003_EMP017_EMP068_EMP012_aggregated.txt
User: user_59, File: encrypted_user_59_auth_EMP057_EMP012_EMP074_EMP011_EMP073_EMP046_EMP050_aggregated.txt
User: user_89, File: encrypted_user_89_auth_EMP012_EMP055_aggregated.txt
User: user_99, File: encrypted_user_99_auth_EMP066_EMP036_EMP012_EMP060_EMP044_EMP018_EMP035_EMP009_aggregated.txt
User: user_9, File: encrypted_user_9_auth_EMP051_EMP002_EMP012_EMP043_EMP005_aggregated.txt
User: user_15, File: key_user_15_auth_EMP072_EMP081_EMP056_EMP040_EMP016_EMP039_EMP012_EMP059_EMP076_EMP078_aggregated.txt.bin
User: user_52, File: key_user_52_auth_EMP075_EMP022_EMP038_EMP012_EMP024_EMP060_EMP042_EMP040_a

# 2nd optimiztion

In [4]:
import os
import time
from neo4j import GraphDatabase

class GraphDBManager:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.create_constraints_and_indexes()

    def close(self):
        self.driver.close()

    def create_constraints_and_indexes(self):
        with self.driver.session() as session:
            session.execute_write(self._create_user_name_constraint)
            session.execute_write(self._create_file_index)

    @staticmethod
    def _create_user_name_constraint(tx):
        tx.run("CREATE CONSTRAINT user_name_constraint IF NOT EXISTS FOR (u:User) REQUIRE u.name IS UNIQUE")

    @staticmethod
    def _create_file_index(tx):
        tx.run("CREATE INDEX file_name_index IF NOT EXISTS FOR (f:File) ON (f.name)")

    def create_users_with_files(self, folder_paths):
        for folder_path in folder_paths:
            self._process_folder(folder_path)

    def _process_folder(self, folder_path):
        with self.driver.session() as session:
            for file_path in os.listdir(folder_path):
                full_path = os.path.join(folder_path, file_path)
                user_name = self.extract_user_from_filename(file_path)
                session.execute_write(self._create_and_link, user_name, full_path)

    @staticmethod
    def extract_user_from_filename(filename):
        parts = filename.split('_')
        user_index = parts.index('user') + 1
        return 'user_' + parts[user_index]

    @staticmethod
    def _create_and_link(tx, user_name, file_path):
        file_name = os.path.basename(file_path)
        query = """
        MERGE (user:User {name: $user_name})
        MERGE (file:File {name: $file_name, path: $file_path})
        MERGE (user)-[:CONTAINS]->(file)
        """
        tx.run(query, user_name=user_name, file_name=file_name, file_path=file_path)

    def get_files_by_keyword(self, keyword):
        query = """
        MATCH (user:User)-[:CONTAINS]->(file:File)
        WHERE file.name CONTAINS $keyword
        RETURN user.name AS userName, collect(file.name) AS fileNames
        """
        with self.driver.session() as session:
            start_time = time.time()
            result = session.run(query, keyword=keyword)
            records = [(record['userName'], record['fileNames']) for record in result]
            end_time = time.time()
            time_taken = end_time - start_time

            num_users = len(records)
            return records, num_users, time_taken

# Usage example
uri = "neo4j://localhost:7687"
driver = GraphDBManager(uri, "neo4j", "Rohan@556")

folder_paths = [
    "C:/Users/ROHAN/IotSimulation/AES_enc",
    "C:/Users/ROHAN/IotSimulation/AES_Keys"
]
driver.create_users_with_files(folder_paths)

keyword = "EMP012"
files, num_users, time_taken = driver.get_files_by_keyword(keyword)
print(f"Time taken for retrieval: {time_taken} seconds")
print(f"Number of unique users found: {num_users}")
for user, file_names in files:
    print(f"User: {user}, Files: {file_names}")

driver.close()


Time taken for retrieval: 2.2765259742736816 seconds
Number of unique users found: 324
User: user_1013, Files: ['encrypted_user_1013_auth_EMP012_EMP061_EMP045_EMP010_EMP055_EMP077_EMP003_aggregated.txt', 'key_user_1013_auth_EMP012_EMP061_EMP045_EMP010_EMP055_EMP077_EMP003_aggregated.txt.bin']
User: user_1015, Files: ['encrypted_user_1015_auth_EMP056_EMP017_EMP065_EMP060_EMP012_EMP023_EMP072_EMP081_EMP022_aggregated.txt', 'key_user_1015_auth_EMP056_EMP017_EMP065_EMP060_EMP012_EMP023_EMP072_EMP081_EMP022_aggregated.txt.bin']
User: user_1019, Files: ['encrypted_user_1019_auth_EMP058_EMP047_EMP007_EMP006_EMP023_EMP048_EMP012_EMP053_EMP079_aggregated.txt', 'key_user_1019_auth_EMP058_EMP047_EMP007_EMP006_EMP023_EMP048_EMP012_EMP053_EMP079_aggregated.txt.bin']
User: user_1048, Files: ['encrypted_user_1048_auth_EMP058_EMP026_EMP012_EMP054_EMP035_aggregated.txt', 'key_user_1048_auth_EMP058_EMP026_EMP012_EMP054_EMP035_aggregated.txt.bin']
User: user_1104, Files: ['encrypted_user_1104_auth_EMP023

# mySQL

In [2]:
import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

# Replace '3306' with 'localhost' for the host parameter
host = "localhost"
username = "root"
password = ""
database_name = "trial1"

# Create a connection to the database
connection = create_connection(host, username, password, database_name)

# Remember to close the connection when done
if connection and connection.is_connected():
    connection.close()
    print("MySQL connection is closed")


Connection to MySQL DB successful
MySQL connection is closed


In [7]:
import os
import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

def execute_query(connection, query, data_tuple=None):
    cursor = connection.cursor()
    try:
        if data_tuple:
            cursor.execute(query, data_tuple)
        else:
            cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

def create_user_files_table(connection):
    create_table_query = """
    CREATE TABLE IF NOT EXISTS UserFiles (
        UserID INT AUTO_INCREMENT PRIMARY KEY,
        UserName VARCHAR(255) NOT NULL UNIQUE,
        EncryptedFile VARCHAR(255),
        KeyFile VARCHAR(255)
    );
    """
    execute_query(connection, create_table_query)

def insert_user_file_data(connection, user_name, encrypted_file, key_file):
    insert_data_query = """
    INSERT INTO UserFiles (UserName, EncryptedFile, KeyFile)
    VALUES (%s, %s, %s)
    ON DUPLICATE KEY UPDATE EncryptedFile = VALUES(EncryptedFile), KeyFile = VALUES(KeyFile);
    """
    data_tuple = (user_name, encrypted_file, key_file)
    execute_query(connection, insert_data_query, data_tuple)

def fetch_all_user_file_data(connection):
    fetch_query = "SELECT * FROM UserFiles;"
    cursor = connection.cursor()
    try:
        cursor.execute(fetch_query)
        records = cursor.fetchall()
        print("Fetched all data from UserFiles table:")
        for row in records:
            print(row)
    except Error as e:
        print(f"The error '{e}' occurred")

def extract_user_from_filename(filename):
    parts = filename.split('_')
    user_index = parts.index('user') + 1
    return '_'.join(parts[user_index - 1:user_index + 1])

def find_key_file_for_user(key_files, user_name):
    for key_file in key_files:
        if user_name in key_file:
            return key_file
    return None

def process_files_and_store_info(encrypted_files_dir, key_files_dir, connection):
    encrypted_files = read_files_from_directory(encrypted_files_dir)
    key_files = read_files_from_directory(key_files_dir)

    print(f"Encrypted files found: {encrypted_files}")
    print(f"Key files found: {key_files}")

    for encrypted_file_name in encrypted_files:
        if encrypted_file_name.endswith('.txt'):
            user_name = extract_user_from_filename(encrypted_file_name)
            key_file_name = find_key_file_for_user(key_files, user_name)
            if key_file_name:
                print(f"Inserting data for {user_name}")
                insert_user_file_data(connection, user_name, encrypted_file_name, key_file_name)
            else:
                print(f"Key file for {user_name} not found.")


def read_files_from_directory(directory_path):
    try:
        return os.listdir(directory_path)
    except FileNotFoundError:
        print(f"The directory {directory_path} does not exist.")
        return []

# Database connection details
host = "localhost"
username = "root"
password = ""
database_name = "trial2"

# Establish database connection
connection = create_connection(host, username, password, database_name)

# Create the UserFiles table
create_user_files_table(connection)

# Directory paths for the files
# Corrected Directory paths
encrypted_files_dir = "C:/Users/ROHAN/IotSimulation/AES_enc"  # Path to your .txt files directory
key_files_dir = "C:/Users/ROHAN/IotSimulation/AES_Keys"  # Path to your key files directory

# Process files and store the information in the database
process_files_and_store_info(encrypted_files_dir, key_files_dir, connection)

# Fetch and display all data from the UserFiles table
fetch_all_user_file_data(connection)

# Close the database connection
if connection.is_connected():
    connection.close()
    print("MySQL connection is closed")


Connection to MySQL DB successful
Query executed successfully
Encrypted files found: ['encrypted_user_1000_auth_EMP040_EMP047_EMP024_EMP034_EMP032_EMP063_EMP018_aggregated.txt', 'encrypted_user_1001_auth_EMP080_EMP059_EMP074_EMP061_EMP014_EMP011_aggregated.txt', 'encrypted_user_1002_auth_EMP010_EMP066_EMP008_EMP033_EMP049_EMP016_EMP019_EMP029_aggregated.txt', 'encrypted_user_1003_auth_EMP059_EMP069_EMP053_EMP062_EMP039_EMP071_aggregated.txt', 'encrypted_user_1004_auth_EMP002_EMP021_EMP031_EMP067_EMP066_EMP051_EMP083_EMP013_aggregated.txt', 'encrypted_user_1005_auth_EMP083_EMP038_aggregated.txt', 'encrypted_user_1006_auth_EMP048_EMP023_EMP021_EMP077_EMP014_EMP039_EMP031_aggregated.txt', 'encrypted_user_1007_auth_EMP055_EMP032_EMP039_EMP044_EMP068_EMP009_EMP006_EMP067_EMP019_aggregated.txt', 'encrypted_user_1008_auth_EMP031_aggregated.txt', 'encrypted_user_1009_auth_EMP021_EMP061_aggregated.txt', 'encrypted_user_100_auth_EMP048_EMP050_EMP069_EMP035_EMP010_EMP070_EMP071_EMP057_EMP013_aggr

Query executed successfully
Inserting data for user_1041
Query executed successfully
Inserting data for user_1042
Query executed successfully
Inserting data for user_1043
Query executed successfully
Inserting data for user_1044
Query executed successfully
Inserting data for user_1045
Query executed successfully
Inserting data for user_1046
Query executed successfully
Inserting data for user_1047
Query executed successfully
Inserting data for user_1048
Query executed successfully
Inserting data for user_1049
Query executed successfully
Inserting data for user_104
Query executed successfully
Inserting data for user_1050
Query executed successfully
Inserting data for user_1051
Query executed successfully
Inserting data for user_1052
Query executed successfully
Inserting data for user_1053
Query executed successfully
Inserting data for user_1054
Query executed successfully
Inserting data for user_1055
Query executed successfully
Inserting data for user_1056
Query executed successfully
Inse

Query executed successfully
Inserting data for user_1193
Query executed successfully
Inserting data for user_1194
Query executed successfully
Inserting data for user_1195
Query executed successfully
Inserting data for user_1196
Query executed successfully
Inserting data for user_1197
Query executed successfully
Inserting data for user_1198
Query executed successfully
Inserting data for user_1199
Query executed successfully
Inserting data for user_119
Query executed successfully
Inserting data for user_11
Query executed successfully
Inserting data for user_1200
Query executed successfully
Inserting data for user_1201
Query executed successfully
Inserting data for user_1202
Query executed successfully
Inserting data for user_1203
Query executed successfully
Inserting data for user_1204
Query executed successfully
Inserting data for user_1205
Query executed successfully
Inserting data for user_1206
Query executed successfully
Inserting data for user_1207
Query executed successfully
Insert

Query executed successfully
Inserting data for user_1398
Query executed successfully
Inserting data for user_1399
Query executed successfully
Inserting data for user_139
Query executed successfully
Inserting data for user_13
Query executed successfully
Inserting data for user_1400
Query executed successfully
Inserting data for user_1401
Query executed successfully
Inserting data for user_1402
Query executed successfully
Inserting data for user_1403
Query executed successfully
Inserting data for user_1404
Query executed successfully
Inserting data for user_1405
Query executed successfully
Inserting data for user_1406
Query executed successfully
Inserting data for user_1407
Query executed successfully
Inserting data for user_1408
Query executed successfully
Inserting data for user_1409
Query executed successfully
Inserting data for user_140
Query executed successfully
Inserting data for user_1410
Query executed successfully
Inserting data for user_1411
Query executed successfully
Inserti

Query executed successfully
Inserting data for user_1564
Query executed successfully
Inserting data for user_1565
Query executed successfully
Inserting data for user_1566
Query executed successfully
Inserting data for user_1567
Query executed successfully
Inserting data for user_1568
Query executed successfully
Inserting data for user_1569
Query executed successfully
Inserting data for user_156
Query executed successfully
Inserting data for user_1570
Query executed successfully
Inserting data for user_1571
Query executed successfully
Inserting data for user_1572
Query executed successfully
Inserting data for user_1573
Query executed successfully
Inserting data for user_1574
Query executed successfully
Inserting data for user_1575
Query executed successfully
Inserting data for user_1576
Query executed successfully
Inserting data for user_1577
Query executed successfully
Inserting data for user_1578
Query executed successfully
Inserting data for user_1579
Query executed successfully
Inse

...

Query executed successfully
Inserting data for user_468
Query executed successfully
Inserting data for user_4690
Query executed successfully
Inserting data for user_4691
Query executed successfully
Inserting data for user_4692
Query executed successfully
Inserting data for user_4693
Query executed successfully
Inserting data for user_4694
Query executed successfully
Inserting data for user_4695
Query executed successfully
Inserting data for user_4696
Query executed successfully
Inserting data for user_4697
Query executed successfully
Inserting data for user_4698
Query executed successfully
Inserting data for user_4699
Query executed successfully
Inserting data for user_469
Query executed successfully
Inserting data for user_46
Query executed successfully
Inserting data for user_4700
Query executed successfully
Inserting data for user_4701
Query executed successfully
Inserting data for user_4702
Query executed successfully
Inserting data for user_4703
Query executed successfully
Inserti

Query executed successfully
Inserting data for user_4867
Query executed successfully
Inserting data for user_4868
Query executed successfully
Inserting data for user_4869
Query executed successfully
Inserting data for user_486
Query executed successfully
Inserting data for user_4870
Query executed successfully
Inserting data for user_4871
Query executed successfully
Inserting data for user_4872
Query executed successfully
Inserting data for user_4873
Query executed successfully
Inserting data for user_4874
Query executed successfully
Inserting data for user_4875
Query executed successfully
Inserting data for user_4876
Query executed successfully
Inserting data for user_4877
Query executed successfully
Inserting data for user_4878
Query executed successfully
Inserting data for user_4879
Query executed successfully
Inserting data for user_487
Query executed successfully
Inserting data for user_4880
Query executed successfully
Inserting data for user_4881
Query executed successfully
Inser

Query executed successfully
Inserting data for user_533
Query executed successfully
Inserting data for user_534
Query executed successfully
Inserting data for user_535
Query executed successfully
Inserting data for user_536
Query executed successfully
Inserting data for user_537
Query executed successfully
Inserting data for user_538
Query executed successfully
Inserting data for user_539
Query executed successfully
Inserting data for user_53
Query executed successfully
Inserting data for user_540
Query executed successfully
Inserting data for user_541
Query executed successfully
Inserting data for user_542
Query executed successfully
Inserting data for user_543
Query executed successfully
Inserting data for user_544
Query executed successfully
Inserting data for user_545
Query executed successfully
Inserting data for user_546
Query executed successfully
Inserting data for user_547
Query executed successfully
Inserting data for user_548
Query executed successfully
Inserting data for us

Query executed successfully
Inserting data for user_679
Query executed successfully
Inserting data for user_67
Query executed successfully
Inserting data for user_680
Query executed successfully
Inserting data for user_681
Query executed successfully
Inserting data for user_682
Query executed successfully
Inserting data for user_683
Query executed successfully
Inserting data for user_684
Query executed successfully
Inserting data for user_685
Query executed successfully
Inserting data for user_686
Query executed successfully
Inserting data for user_687
Query executed successfully
Inserting data for user_688
Query executed successfully
Inserting data for user_689
Query executed successfully
Inserting data for user_68
Query executed successfully
Inserting data for user_690
Query executed successfully
Inserting data for user_691
Query executed successfully
Inserting data for user_692
Query executed successfully
Inserting data for user_693
Query executed successfully
Inserting data for use

Query executed successfully
Inserting data for user_853
Query executed successfully
Inserting data for user_854
Query executed successfully
Inserting data for user_855
Query executed successfully
Inserting data for user_856
Query executed successfully
Inserting data for user_857
Query executed successfully
Inserting data for user_858
Query executed successfully
Inserting data for user_859
Query executed successfully
Inserting data for user_85
Query executed successfully
Inserting data for user_860
Query executed successfully
Inserting data for user_861
Query executed successfully
Inserting data for user_862
Query executed successfully
Inserting data for user_863
Query executed successfully
Inserting data for user_864
Query executed successfully
Inserting data for user_865
Query executed successfully
Inserting data for user_866
Query executed successfully
Inserting data for user_867
Query executed successfully
Inserting data for user_868
Query executed successfully
Inserting data for us

In [ ]:
C:/Users/ROHAN/IotSimulation/AES_Keys

In [ ]:
C:/Users/ROHAN/IotSimulation/AES_enc

In [8]:
import mysql.connector
from mysql.connector import Error
import time

class MySQLDBManager:
    def __init__(self, host, user, password, db):
        self.connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=db
        )
        if self.connection.is_connected():
            print("Connection to MySQL DB successful")

    def close(self):
        if self.connection.is_connected():
            self.connection.close()
            print("MySQL connection is closed")

    def get_users_and_files_by_keyword(self, keyword):
        query = """
        SELECT UserName, EncryptedFile, KeyFile
        FROM UserFiles
        WHERE EncryptedFile LIKE %s OR KeyFile LIKE %s
        """
        cursor = self.connection.cursor()
        
        start_time = time.time()  # Start timing here
        cursor.execute(query, ('%' + keyword + '%', '%' + keyword + '%'))
        result = cursor.fetchall()
        end_time = time.time()  # End timing here
        
        time_taken = end_time - start_time  # Calculate the time taken
        
        unique_users = set([row[0] for row in result])
        return result, len(unique_users), time_taken

# Usage example
host = "localhost"
user = "root"
password = ""
db_name = "trial2"

db_manager = MySQLDBManager(host, user, password, db_name)

# Assuming you've already stored users and their files
users_files, user_count, time_taken = db_manager.get_users_and_files_by_keyword("EMP012")
print(f"Time taken: {time_taken} seconds")
print(f"Number of unique users with 'EMP058' in their filenames: {user_count}")
for user, encrypted_file, key_file in users_files:
    print(f"User: {user}, Encrypted File: {encrypted_file}, Key File: {key_file}")

db_manager.close()


Connection to MySQL DB successful
Time taken: 0.018001794815063477 seconds
Number of unique users with 'EMP058' in their filenames: 355
User: user_1013, Encrypted File: encrypted_user_1013_auth_EMP012_EMP061_EMP045_EMP010_EMP055_EMP077_EMP003_aggregated.txt, Key File: key_user_1013_auth_EMP012_EMP061_EMP045_EMP010_EMP055_EMP077_EMP003_aggregated.txt.bin
User: user_1015, Encrypted File: encrypted_user_1015_auth_EMP056_EMP017_EMP065_EMP060_EMP012_EMP023_EMP072_EMP081_EMP022_aggregated.txt, Key File: key_user_1015_auth_EMP056_EMP017_EMP065_EMP060_EMP012_EMP023_EMP072_EMP081_EMP022_aggregated.txt.bin
User: user_1019, Encrypted File: encrypted_user_1019_auth_EMP058_EMP047_EMP007_EMP006_EMP023_EMP048_EMP012_EMP053_EMP079_aggregated.txt, Key File: key_user_1019_auth_EMP058_EMP047_EMP007_EMP006_EMP023_EMP048_EMP012_EMP053_EMP079_aggregated.txt.bin
User: user_1048, Encrypted File: encrypted_user_1048_auth_EMP058_EMP026_EMP012_EMP054_EMP035_aggregated.txt, Key File: key_user_1048_auth_EMP058_EMP

In [14]:
import mysql.connector
from mysql.connector import Error
import time

class MySQLDBManager:
    def __init__(self, host, user, password, db):
        self.connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=db
        )
        if self.connection.is_connected():
            print("Connection to MySQL DB successful")

    def close(self):
        if self.connection.is_connected():
            self.connection.close()
            print("MySQL connection is closed")

    def search_files_by_keyword(self, keyword):
        query = """
        SELECT UserName, EncryptedFile, KeyFile
        FROM UserFiles
        WHERE EncryptedFile LIKE %s OR KeyFile LIKE %s
        """
        cursor = self.connection.cursor()
        
        start_time = time.time()
        cursor.execute(query, ('%' + keyword + '%', '%' + keyword + '%'))
        result = cursor.fetchall()
        end_time = time.time()
        
        time_taken = end_time - start_time
        
        unique_users = set([row[0] for row in result])
        return result, len(unique_users), time_taken

# Usage example for MySQL
host = "localhost"
user = "root"
password = ""
db_name = "trial2"

db_manager_mysql = MySQLDBManager(host, user, password, db_name)

keyword = "EMP017"
mysql_files, mysql_user_count, mysql_time_taken = db_manager_mysql.search_files_by_keyword(keyword)
print(f"Time taken for MySQL retrieval: {mysql_time_taken} seconds")
print(f"Number of unique users with '{keyword}' in their filenames (MySQL): {mysql_user_count}")
for user, encrypted_file, key_file in mysql_files:
    print(f"User: {user}, Encrypted File: {encrypted_file}, Key File: {key_file}")

db_manager_mysql.close()


Connection to MySQL DB successful
Time taken for MySQL retrieval: 0.03519415855407715 seconds
Number of unique users with 'EMP017' in their filenames (MySQL): 381
User: user_1012, Encrypted File: encrypted_user_1012_auth_EMP015_EMP007_EMP044_EMP069_EMP017_EMP051_EMP010_EMP026_aggregated.txt, Key File: key_user_1012_auth_EMP015_EMP007_EMP044_EMP069_EMP017_EMP051_EMP010_EMP026_aggregated.txt.bin
User: user_1015, Encrypted File: encrypted_user_1015_auth_EMP056_EMP017_EMP065_EMP060_EMP012_EMP023_EMP072_EMP081_EMP022_aggregated.txt, Key File: key_user_1015_auth_EMP056_EMP017_EMP065_EMP060_EMP012_EMP023_EMP072_EMP081_EMP022_aggregated.txt.bin
User: user_1039, Encrypted File: encrypted_user_1039_auth_EMP017_EMP052_EMP054_EMP029_EMP071_aggregated.txt, Key File: key_user_1039_auth_EMP017_EMP052_EMP054_EMP029_EMP071_aggregated.txt.bin
User: user_1052, Encrypted File: encrypted_user_1052_auth_EMP076_EMP056_EMP034_EMP083_EMP082_EMP024_EMP065_EMP052_EMP058_EMP017_aggregated.txt, Key File: key_user_

In [13]:
import os
import time
from neo4j import GraphDatabase

class GraphDBManager:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.create_constraints_and_indexes()

    def close(self):
        self.driver.close()

    def create_constraints_and_indexes(self):
        with self.driver.session() as session:
            session.execute_write(self._create_user_name_constraint)
            session.execute_write(self._create_file_index)

    @staticmethod
    def _create_user_name_constraint(tx):
        tx.run("CREATE CONSTRAINT user_name_constraint IF NOT EXISTS FOR (u:User) REQUIRE u.name IS UNIQUE")

    @staticmethod
    def _create_file_index(tx):
        tx.run("CREATE INDEX file_name_index IF NOT EXISTS FOR (f:File) ON (f.name)")

    def search_files_by_keyword(self, keyword):
        query = """
        MATCH (user:User)-[:CONTAINS]->(file:File)
        WHERE file.name CONTAINS $keyword
        RETURN user.name AS userName, file.name AS fileName
        """
        with self.driver.session() as session:
            start_time = time.time()
            result = session.run(query, keyword=keyword)
            records = [record for record in result]
            end_time = time.time()
            time_taken = end_time - start_time
            return records, time_taken

# Usage example for Neo4j
uri = "neo4j://localhost:7687"
user = "neo4j"
password = "Rohan@556"

db_manager_neo4j = GraphDBManager(uri, user, password)

keyword = "EMP017"
neo4j_files, neo4j_time_taken = db_manager_neo4j.search_files_by_keyword(keyword)
print(f"Time taken for Neo4j retrieval: {neo4j_time_taken} seconds")
for record in neo4j_files:
    print(f"User: {record['userName']}, File: {record['fileName']}")

db_manager_neo4j.close()


Time taken for Neo4j retrieval: 2.120929002761841 seconds


# ----------------------------------------------

# mySQL

In [19]:
import mysql.connector
import os
import time

# Function to create MySQL connection
def create_mysql_connection(host_name, user_name, user_password, db_name):
    connection = mysql.connector.connect(
        host=host_name,
        user=user_name,
        passwd=user_password,
        database=db_name
    )
    return connection

# Function to insert data into MySQL
def insert_user_file_data_mysql(connection, user_name, encrypted_file, key_file):
    cursor = connection.cursor()
    query = """
    INSERT INTO UserFiles (UserName, EncryptedFile, KeyFile)
    VALUES (%s, %s, %s)
    ON DUPLICATE KEY UPDATE EncryptedFile = VALUES(EncryptedFile), KeyFile = VALUES(KeyFile);
    """
    cursor.execute(query, (user_name, encrypted_file, key_file))
    connection.commit()

# Extract user name from file name
def extract_user_from_filename(filename):
    parts = filename.split('_')
    user_index = parts.index('user') + 1
    return 'user_' + parts[user_index]

# Connect to MySQL
connection = create_mysql_connection('localhost', 'root', '', 'trial4')



# Assume you have two directories containing the respective files
encrypted_files_dir = "C:/Users/ROHAN/IotSimulation/AES_enc"
key_files_dir = "C:/Users/ROHAN/IotSimulation/AES_Keys"

# Process and insert data into MySQL
for file in os.listdir(encrypted_files_dir):
    user_name = extract_user_from_filename(file)
    encrypted_file = file
    key_file = 'key_' + file  # Assuming a naming convention
    insert_user_file_data_mysql(connection, user_name, encrypted_file, key_file)

# Measure retrieval performance in MySQL
start_time = time.time()
cursor = connection.cursor()
cursor.execute("SELECT * FROM UserFiles WHERE UserName = 'user_1'")
result = cursor.fetchall()
end_time = time.time()
print(f"Time taken for MySQL retrieval: {end_time - start_time} seconds")

# Close MySQL connection
connection.close()


Time taken for MySQL retrieval: 0.0059947967529296875 seconds


In [24]:
import mysql.connector
import os
import time

# Function to create MySQL connection
def create_mysql_connection(host_name, user_name, user_password, db_name):
    connection = mysql.connector.connect(
        host=host_name,
        user=user_name,
        passwd=user_password,
        database=db_name
    )
    return connection

# Function to create constraints and indexes (MySQL doesn't have native constraints and indexes like Neo4j)
def create_mysql_constraints_and_indexes(connection):
    cursor = connection.cursor()
    
    # Create a UNIQUE constraint on UserName (if not already done)
    cursor.execute("CREATE TABLE IF NOT EXISTS UserFiles (ID INT AUTO_INCREMENT PRIMARY KEY, UserName VARCHAR(255) UNIQUE, EncryptedFile VARCHAR(255), KeyFile VARCHAR(255))")
    connection.commit()

# Function to insert data into MySQL
def insert_user_file_data_mysql(connection, user_name, encrypted_file, key_file):
    cursor = connection.cursor()
    query = """
    INSERT INTO UserFiles (UserName, EncryptedFile, KeyFile)
    VALUES (%s, %s, %s)
    ON DUPLICATE KEY UPDATE EncryptedFile = VALUES(EncryptedFile), KeyFile = VALUES(KeyFile);
    """
    cursor.execute(query, (user_name, encrypted_file, key_file))
    connection.commit()

# Extract user name from file name
def extract_user_from_filename(filename):
    parts = filename.split('_')
    user_index = parts.index('user') + 1
    return 'user_' + parts[user_index]

# Connect to MySQL
connection = create_mysql_connection('localhost', 'root', '', 'trial5')

# Create constraints and indexes (not native in MySQL)
create_mysql_constraints_and_indexes(connection)

# Assume you have two directories containing the respective files
encrypted_files_dir = "C:/Users/ROHAN/IotSimulation/AES_enc"
key_files_dir = "C:/Users/ROHAN/IotSimulation/AES_Keys"

# Process and insert data into MySQL
for file in os.listdir(encrypted_files_dir):
    user_name = extract_user_from_filename(file)
    encrypted_file = file
    key_file = 'key_' + file  # Assuming a naming convention
    insert_user_file_data_mysql(connection, user_name, encrypted_file, key_file)

# Measure retrieval performance in MySQL
start_time = time.time()
cursor = connection.cursor()
cursor.execute("SELECT * FROM UserFiles WHERE UserName = 'user_15'")
result = cursor.fetchall()
end_time = time.time()
print(f"Time taken for MySQL retrieval: {end_time - start_time} seconds")

# Close MySQL connection
connection.close()


Time taken for MySQL retrieval: 0.002995014190673828 seconds


# NEO

In [17]:
import os
from neo4j import GraphDatabase

class GraphDBManager:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_constraints_and_indexes(self):
        with self.driver.session() as session:
            session.run("CREATE CONSTRAINT user_name_constraint IF NOT EXISTS FOR (u:User) REQUIRE u.name IS UNIQUE")
            session.run("CREATE INDEX file_name_index IF NOT EXISTS FOR (f:File) ON (f.name)")

    def create_and_link_files(self, user_name, file_name):
        with self.driver.session() as session:
            query = """
            MERGE (user:User {name: $user_name})
            MERGE (file:File {name: $file_name})
            MERGE (user)-[:CONTAINS]->(file)
            """
            session.run(query, user_name=user_name, file_name=file_name)

    def extract_user_from_filename(self, filename):
        parts = filename.split('_')
        if 'user' in parts:
            user_index = parts.index('user') + 1
            return 'user_' + parts[user_index]
        return None

# Initialize GraphDBManager
uri = "neo4j://localhost:7687"
user = "neo4j"
password = "Rohan@556"
graph_db = GraphDBManager(uri, user, password)

# Create constraints and indexes
graph_db.create_constraints_and_indexes()

# Directories containing the files
encrypted_files_dir = "C:/Users/ROHAN/IotSimulation/AES_enc"
key_files_dir = "C:/Users/ROHAN/IotSimulation/AES_Keys"

# Process and upload files to Neo4j
for directory in [encrypted_files_dir, key_files_dir]:
    for file_name in os.listdir(directory):
        user_name = graph_db.extract_user_from_filename(file_name)
        if user_name:
            graph_db.create_and_link_files(user_name, file_name)

# Close connection
graph_db.close()


# retrival

In [20]:
import mysql.connector
import time

# Create MySQL connection
def create_mysql_connection(host_name, user_name, user_password, db_name):
    return mysql.connector.connect(
        host=host_name,
        user=user_name,
        passwd=user_password,
        database=db_name
    )

# Fetch user files with keyword
def fetch_user_files_with_keyword_mysql(connection, keyword):
    query = """
    SELECT UserName, EncryptedFile, KeyFile
    FROM UserFiles
    WHERE EncryptedFile LIKE %s OR KeyFile LIKE %s
    """
    start_time = time.time()
    cursor = connection.cursor()
    cursor.execute(query, ('%' + keyword + '%', '%' + keyword + '%'))
    result = cursor.fetchall()
    end_time = time.time()
    time_taken = end_time - start_time

    # Count total number of files
    file_count = sum(1 for row in result for file in row[1:] if keyword in file)
    return result, file_count, time_taken

# Connect to MySQL
connection = create_mysql_connection('localhost', 'root', '', 'trial3')

# Fetch and display data
keyword = "EMP060"
files, file_count, time_taken = fetch_user_files_with_keyword_mysql(connection, keyword)
print(f"Time taken for MySQL retrieval: {time_taken} seconds")
print(f"Total number of files: {file_count}")
for row in files:
    print(row)

# Close MySQL connection
connection.close()


Time taken for MySQL retrieval: 0.046996355056762695 seconds
Total number of files: 598
('user_1015', 'encrypted_user_1015_auth_EMP056_EMP017_EMP065_EMP060_EMP012_EMP023_EMP072_EMP081_EMP022_aggregated.txt', 'key_encrypted_user_1015_auth_EMP056_EMP017_EMP065_EMP060_EMP012_EMP023_EMP072_EMP081_EMP022_aggregated.txt')
('user_1070', 'encrypted_user_1070_auth_EMP060_EMP044_aggregated.txt', 'key_encrypted_user_1070_auth_EMP060_EMP044_aggregated.txt')
('user_1101', 'encrypted_user_1101_auth_EMP060_EMP032_EMP048_EMP025_EMP078_EMP001_aggregated.txt', 'key_encrypted_user_1101_auth_EMP060_EMP032_EMP048_EMP025_EMP078_EMP001_aggregated.txt')
('user_1103', 'encrypted_user_1103_auth_EMP060_EMP050_EMP045_aggregated.txt', 'key_encrypted_user_1103_auth_EMP060_EMP050_EMP045_aggregated.txt')
('user_1173', 'encrypted_user_1173_auth_EMP034_EMP060_EMP081_aggregated.txt', 'key_encrypted_user_1173_auth_EMP034_EMP060_EMP081_aggregated.txt')
('user_1175', 'encrypted_user_1175_auth_EMP078_EMP077_EMP040_EMP026_EM

In [22]:
from neo4j import GraphDatabase
import time

# Define a class to manage Neo4j database operations
class Neo4jManager:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def fetch_user_files_with_keyword_neo4j(self, keyword):
        query = """
        MATCH (user:User)-[:CONTAINS]->(file:File)
        WHERE file.name CONTAINS $keyword
        RETURN user.name AS userName, collect(file.name) AS fileNames
        """
        start_time = time.time()
        with self.driver.session() as session:
            result = session.run(query, keyword=keyword)
            records = [record for record in result]
        end_time = time.time()
        time_taken = end_time - start_time
        return records, time_taken

# Connect to Neo4j
uri = "neo4j://localhost:7687"
user = "neo4j"
password = "Rohan@556"  # Replace with your Neo4j password
neo4j_db = Neo4jManager(uri, user, password)

# Fetch and display data
keyword = "EMP060"
records, time_taken = neo4j_db.fetch_user_files_with_keyword_neo4j(keyword)
print(f"Time taken for Neo4j retrieval: {time_taken} seconds")
print(f"Total number of records: {len(records)}")
for record in records:
    print(f"User Name: {record['userName']}, File Names: {record['fileNames']}")

# Close Neo4j connection
neo4j_db.close()


Time taken for Neo4j retrieval: 4.15678858757019 seconds
Total number of records: 299
User Name: user_1015, File Names: ['encrypted_user_1015_auth_EMP056_EMP017_EMP065_EMP060_EMP012_EMP023_EMP072_EMP081_EMP022_aggregated.txt', 'key_user_1015_auth_EMP056_EMP017_EMP065_EMP060_EMP012_EMP023_EMP072_EMP081_EMP022_aggregated.txt.bin']
User Name: user_1070, File Names: ['encrypted_user_1070_auth_EMP060_EMP044_aggregated.txt', 'key_user_1070_auth_EMP060_EMP044_aggregated.txt.bin']
User Name: user_1101, File Names: ['encrypted_user_1101_auth_EMP060_EMP032_EMP048_EMP025_EMP078_EMP001_aggregated.txt', 'key_user_1101_auth_EMP060_EMP032_EMP048_EMP025_EMP078_EMP001_aggregated.txt.bin']
User Name: user_1103, File Names: ['encrypted_user_1103_auth_EMP060_EMP050_EMP045_aggregated.txt', 'key_user_1103_auth_EMP060_EMP050_EMP045_aggregated.txt.bin']
User Name: user_1173, File Names: ['encrypted_user_1173_auth_EMP034_EMP060_EMP081_aggregated.txt', 'key_user_1173_auth_EMP034_EMP060_EMP081_aggregated.txt.bin

In [25]:
from neo4j import GraphDatabase
import time

# Define a class to manage Neo4j database operations
class Neo4jManager:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def search_files_by_keyword(self, keyword):
        query = """
        MATCH (user:User)-[:CONTAINS]->(file:File)
        WHERE file.name CONTAINS $keyword
        RETURN user.name AS userName, collect(file.name) AS fileNames
        """
        start_time = time.time()
        with self.driver.session() as session:
            result = session.run(query, keyword=keyword)
            records = [record for record in result]
        end_time = time.time()
        time_taken = end_time - start_time
        return records, time_taken

# Connect to Neo4j
uri = "neo4j://localhost:7687"
user = "neo4j"
password = "Rohan@556"  # Replace with your Neo4j password
neo4j_db = Neo4jManager(uri, user, password)

# Search and display data
keyword = "EMP060"
records, time_taken = neo4j_db.search_files_by_keyword(keyword)
print(f"Time taken for Neo4j search: {time_taken} seconds")
print(f"Total number of records: {len(records)}")
for record in records:
    print(f"User Name: {record['userName']}, File Names: {record['fileNames']}")

# Close Neo4j connection
neo4j_db.close()


Time taken for Neo4j search: 4.117011785507202 seconds
Total number of records: 299
User Name: user_1015, File Names: ['encrypted_user_1015_auth_EMP056_EMP017_EMP065_EMP060_EMP012_EMP023_EMP072_EMP081_EMP022_aggregated.txt', 'key_user_1015_auth_EMP056_EMP017_EMP065_EMP060_EMP012_EMP023_EMP072_EMP081_EMP022_aggregated.txt.bin']
User Name: user_1070, File Names: ['encrypted_user_1070_auth_EMP060_EMP044_aggregated.txt', 'key_user_1070_auth_EMP060_EMP044_aggregated.txt.bin']
User Name: user_1101, File Names: ['encrypted_user_1101_auth_EMP060_EMP032_EMP048_EMP025_EMP078_EMP001_aggregated.txt', 'key_user_1101_auth_EMP060_EMP032_EMP048_EMP025_EMP078_EMP001_aggregated.txt.bin']
User Name: user_1103, File Names: ['encrypted_user_1103_auth_EMP060_EMP050_EMP045_aggregated.txt', 'key_user_1103_auth_EMP060_EMP050_EMP045_aggregated.txt.bin']
User Name: user_1173, File Names: ['encrypted_user_1173_auth_EMP034_EMP060_EMP081_aggregated.txt', 'key_user_1173_auth_EMP034_EMP060_EMP081_aggregated.txt.bin']